# Evaluate regression model

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecay import database
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import run

In [ ]:
df_regression = database.load(rolldecay_table_name='rolldecay_quadratic_b_regression', 
                              limit_score=-np.inf)

df_ikeda = database.load(rolldecay_table_name='rolldecay_simplified_ikeda', 
                              limit_score=-np.inf, exclude_table_name='rolldecay_exclude')

df_regression['g']=9.81
df_regression['rho']=1000
df_regression['Disp']=df_regression['Volume']/(df_regression['scale_factor']**3)
df_regression['B_1_hat'] = run(function=lambdas.B_1_hat_lambda, inputs=df_regression)
df_regression['B_2_hat'] = run(function=lambdas.B_2_hat_lambda, inputs=df_regression)

df_ikeda['g']=9.81
df_ikeda['rho']=1000
df_ikeda['Disp']=df_ikeda['Volume']/(df_ikeda['scale_factor']**3)
df_ikeda['B_1_hat'] = run(function=lambdas.B_1_hat_lambda, inputs=df_ikeda)
df_ikeda['B_2_hat'] = run(function=lambdas.B_2_hat_lambda, inputs=df_ikeda)



In [ ]:
df_regression.describe()

In [ ]:
fig,ax=plt.subplots()
df_regression['score'].hist(bins=40, ax=ax)

In [ ]:
fig,ax=plt.subplots()
df_ikeda['score'].hist(bins=40, ax=ax)

In [ ]:
df_regression['B_1_hat'].hist(bins=40)

In [ ]:
mask = df_regression['B_1_hat'] > df_regression['B_1_hat'].quantile(0.999)
meta_data = df_regression.loc[mask].iloc[0]
meta_data

In [ ]:
print(meta_data['project_path'])

In [ ]:
df_ikeda['B_1_hat'].hist(bins=40)

In [ ]:
df_compare = pd.merge(left=df_regression, right=df_ikeda, how='inner', left_index=True, right_index=True,
                     suffixes=('_regression','_ikeda'))

In [ ]:
interesting = ['score_regression','score_ikeda','B_1_regression','B_1_ikeda','B_2_regression','B_2_ikeda']

In [ ]:
df_compare[interesting].describe()

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='score_ikeda',y='score_regression', ax=ax, style='o', alpha=0.5)

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_1_ikeda',y='B_1_regression', ax=ax, style='o', alpha=0.5)

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_2_ikeda',y='B_2_regression', ax=ax, style='o', alpha=0.5)

In [ ]:
df_regression.describe()

In [ ]:
df_ikeda.describe()

In [ ]:
df_compare.describe()